In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import xarray as xr

In [117]:
# open catalogue dataset
df = pd.read_csv('/home/data/ReAnalysis/ERA5/Storm_analysis/NAECv1/NAEC_1979_2020_v1.csv')

# open catalogue that contains storm that were active for more than 24h
# more in CRCM6 domain
df24 = pd.read_csv('/pampa/cloutier/etc_24_nna.csv')

# open netcdf mask file
file = '/pampa/picart/Masks/mask_GEM5_ERA5grid'
data = xr.open_dataset(file)

# export netcdf to dataframe
mask = data.to_dataframe()

# drop index lat lon, but keep columns
mask = mask.reset_index()

# keep in mask values where HU = true
maskT = mask.loc[mask.HU != False]

### Storm track density

In [2]:
# Define function to calculate distance between two points using Haversine formula
# Haversine package ?
def calculate_distance(lat1, lon1, lat2, lon2):
    # Earth radius in kilometers
    earth_radius = 6371

    # Convert latitude and longitude to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Calculate differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Calculate distance in kilometers
    distance = earth_radius * c

    return distance

In [98]:
df = pd.read_csv('/pampa/cloutier/density.csv')
mam = df.loc[df.season == 'MAM']

In [101]:
# test de la densité avec seulement MAM
# Avec la moyenne pas bonne de 250km
r = 2.25
mam_density = {} # will contain the grid point and density

# iterate through all grid point
for _, row in mam.iterrows() : 
    track = {} #  will contain the year and storm id of the grid points found
    lat1 = row['latitude'] 
    lon1 = row['longitude'] 

    # restricting the research in a 2.5 x 2.5deg square
    neighbors = mam[
        (mam['latitude'] >= lat1 - r) & (mam['latitude'] <= lat1 + r) &
        (mam['longitude'] >= lon1 - r) & (mam['longitude'] <= lon1 + r)
        ]

    # Find the grid points that are within a 250km radius 
    for _, row in neighbors.iterrows():
        lat2 = row['latitude'] 
        lon2 = row['longitude'] 
        storm_count = row['storm_count']
        avgvors = row['avgVORSmax']

        # calculate distance between grid points
        distance = calculate_distance(lat1, lon1, lat2, lon2)

        # add year and storm_id of grid points that are within the radius
        if distance <= 250 : 
            track['storm_count'] = track.get('storm_count', []) + [storm_count]

    # turn track dictionnary into dataframe and count the unique occurence of 
    # every storm for each year
    track_df = pd.DataFrame(data = track)
    #track_df = track_df.groupby('year')['storm_id'].nunique()

    # number of storm per season in average
    average = track_df['storm_count'].mean()

    # add coord and average density
    mam_density['lat'] = mam_density.get('lat', []) + [lat1]
    mam_density['lon'] = mam_density.get('lon', []) + [lon1]
    mam_density['storm_count'] = mam_density.get('storm_count', []) + [average]
    mam_density['avgvors'] = mam_density.get('avgvors', []) + [avgvors]

# transfort m_density into dataframe and save as csv
df_density = pd.DataFrame(data = mam_density)
df_density.to_csv('/pampa/cloutier/test_densite_mam.csv')

In [7]:
import pandas as pd
test = pd.read_csv('/pampa/cloutier/densite_mam_one_gp.csv')

In [8]:
test

,Unnamed: 0,lat,lon,storm_count
0,0,49.0,281.0,6.0
